In [37]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor

# 한글 폰트 설정하기
fe = fm.FontEntry(fname = 'MaruBuri-Regular.otf', name = 'MaruBuri')
fm.fontManager.ttflist.insert(0, fe)
plt.rc('font', family='MaruBuri')

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [30]:
# train, test 데이터를 불러옵니다.
data = pd.read_csv('./train.csv')
test_data = pd.read_csv('./test.csv')

In [40]:
# 결측치 처리
data['배터리용량'].fillna(data['배터리용량'].mean(),inplace=True)
test_data['배터리용량'].fillna(data['배터리용량'].mean(),inplace=True)

<ipython-input-40-7220043876b5>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['배터리용량'].fillna(data['배터리용량'].mean(),inplace=True)
<ipython-input-40-7220043876b5>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', t

In [43]:
# 학습과 예측을 위해 데이터를 분리합니다.
x_train = data.drop(['ID', '가격(백만원)'], axis = 1)
y_train = data['가격(백만원)']

x_test = test_data.drop('ID', axis = 1)

# 범주형 변수에 대해 레이블 인코딩을 적용합니다.
categorical_features = [col for col in x_train.columns if x_train[col].dtype == 'object']

x_train = pd.get_dummies(x_train, columns=categorical_features, drop_first=True)
x_test = pd.get_dummies(x_test, columns=categorical_features, drop_first=True)

# 학습 데이터와 테스트 데이터의 열 불일치 해결
x_test = x_test.reindex(columns=x_train.columns, fill_value=0)


scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# train, test 분리

'''
x_split_train, x_split_test, y_split_train, y_split_test = train_test_split(x_train_scaled, y_train, test_size=0.2, random_state=42)

model = LGBMRegressor(n_estimators=1000, learning_rate=0.05, max_depth=7, random_state=42)
model.fit(x_split_train, y_split_train)
pred = model.predict(x_split_test)

plt.figure(figsize=(10, 6))
ax1 = sns.distplot(y_split_test, hist=False, color="r", label="Actual Value")
ax2 = sns.distplot(pred, hist=False, color="b", label="Fitted Values" , ax=ax1)
plt.title('Actual vs Fitted Values for Test Data')
plt.show()
'''


# 모델 학습
model = LGBMRegressor(n_estimators=1000, learning_rate=0.05, max_depth=7, random_state=42)
model.fit(x_train_scaled, y_train)
pred = model.predict(x_test_scaled)


# sample_submission 파일을 불러옵니다.
submit = pd.read_csv('./sample_submission.csv')

# 예측한 결과를 submit에 저장합니다.
submit['가격(백만원)'] = pred
submit.head()

# 제출을 위해 baseline_submission.csv란 이름으로 CSV 파일을 저장합니다.
submit.to_csv('./baseline_submission.csv',index=False)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003761 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 467
[LightGBM] [Info] Number of data points in the train set: 7497, number of used features: 35
[LightGBM] [Info] Start training from score 62.331949
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
